In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


Открытие файла .xlsx с выбранной страной:

In [3]:
df = pd.read_excel('ITA_NIOT_nov16.xlsx', sheet_name='National IO-tables')

Анализ таблицы и её изменение:

In [4]:
# Выбор года
df = df[(df['Year'] == 2000)]

# Удаление лишних строк
search_for = ['Imports', 'TOT']
mask = df.applymap(lambda x: any(s in str(x) for s in search_for))
rows_analysis = mask.any(axis=1)
rows_to_drop = rows_analysis[rows_analysis == True].index.tolist()
df_filtered = df.drop(rows_to_drop, axis=0)

#Сохранение названий отраслей
description = df_filtered['Description']

# Сохранение данных TOTAL_OUTPUT
total_output = df_filtered['GO']

# Замена нулевых значений в столбце TOTAL_OUTPUT
total_output = (total_output.replace(0, np.inf)).astype(float)

# Удаление лишних столбцов
df_filtered = df_filtered.drop(['GO', 'Year', 'Code','Origin', 'Description','CONS_h', 'CONS_np', 'CONS_g', 'GFCF', 'INVEN', 'EXP'], axis = 1)
#print(df_filtered)
#print(description

Создание матрицы $A = \|{ a_{ij}}\| = \frac{z_{ij}}{Y_{j}}$, где $z_{ij}$ $~-$ поставка продукции $i$-ой отрасли в $j$-ую отрасль в период времени $t$, $Y_{j}$ $~-$ объём производства $j$-ой отрасли в период времени $t$.

In [5]:
# ПРИМЕР
# arr = np.array([[1, 2, 3], [4, 5, 8]]) 
# df = pd.DataFrame(data = arr, index =["row1", "row2"], columns =["col1", "col2", "col3"])
# print(df)
# A = df.copy()
# total_output = df['col3']
# print(total_output)
# for i in df.columns:
    # A[i] = df[i]/total_output
# print(A)

In [6]:
A = df_filtered.copy()
for i in df_filtered.columns:
    A[i] = df_filtered[i]/total_output
#print(A)

Проверка продуктивности (все последовательные главные миноры должны быть положительными):

In [7]:
# ПРИМЕР
# a = np.random.randint(10, size = (3, 3))
# print(a)
# print(np.shape(a)[0])
# for i in range(1, np.shape(a)[0] + 1):
    # print(np.linalg.det(a[:i,:i]))

In [8]:
for i in range(1, np.shape(A)[0] + 1):
    if (np.linalg.det(np.eye(i) - np.matrix(A, dtype = float)[:i,:i]) <= 0):
        print("Условие не выполняется!")
        break
    elif i == np.shape(A)[0]:
        print("Матрица продуктивная!")

Матрица продуктивная!


Считаем связь первого порядка $$d_{i} = \sum\limits_{j=1 \atop {i \neq j}}^n a_{ij}$$ - насколько важна отрасль $i$ при прямых поставках из $i$ в $j$:

In [9]:
d = A.sum(axis = 1) - np.diag(A)
d.index = description
#d

Считаем связь второго порядка $${dd}_{i} = \mathop{\sum\limits_{j=1 \atop {i \neq j}}^n\sum\limits_{k=1}^n a_{ik}a_{kj}}$$ $~-$ как отрасль $i$ косвенно влияет на отрасль $j$:

In [16]:
description.index = A.columns
A.index = description
A.columns = description
A = A.astype(float)
dd = np.zeros((56, 1))
index_to_name = description.copy()
index_to_name.index = np.arange(0, 56)
for i in range(56):
    for j in A.columns:
        for k in A.index:
            if j != k:
                dd[i] = dd[i] + A[index_to_name[i]][k]*A[k][j]
#dd

In [11]:
for i in A.columns:
    A[i] = A[i].apply(lambda x: x if x > 0.02 else 0)
plt.figure(figsize =(15, 10))
G = nx.from_pandas_adjacency(A, create_using = nx.DiGraph)

def w_func(w):
    return w*50
weight = d.copy().apply(w_func)
nx.set_node_attributes(G, weight,'size')

from pyvis.network import Network
net = Network(height='400px', width='800px',heading='', notebook = True, directed=True)
#net.show_buttons(filter_=True)
net.from_nx(G)
net.set_options("""var options = {
  "nodes": {
    "borderWidth": 0,
    "borderWidthSelected": 0,
    "color": "rgba(109, 40, 217)",
    "opacity": null,
    "font": {
      "size": 12
    },
    "scaling": {
      "min": 2,
      "max": 72,
      "label": {
        "enabled": true,
        "min": null,
        "max": null,
        "maxVisible": null,
        "drawThreshold": null
      }
    },
    "shadow": {
      "enabled": true,
      "size": 15,
      "x": 20
    },
    "shape": "circle",
    "size": 80
  },
  "edges": {
    "color": {
      "color": "rgba(196, 181, 253)",
      "opacity": 0.9
    },
    "dashes": true,
    "selfReferenceSize": null,
    "selfReference": {
      "angle": 0.7853981633974483
    },
    "smooth": {
      "forceDirection": "none"
    }
  },
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -2000
    },
    "minVelocity": 0.75
  }
}"""
)
#net.toggle_physics(True) # выключение физического взаимодействия
net.show('mygraph.html')

<Figure size 1080x720 with 0 Axes>

In [12]:
plt.figure(figsize =(15, 10))
dd = pd.DataFrame(dd)
weight = dd.copy().apply(w_func)
nx.set_node_attributes(G,weight,'size')

net2 = Network(height='400px', width='800px',heading='', notebook = True, directed=True)
#net.show_buttons(filter_=True)
net2.from_nx(G)
net2.set_options("""var options = {
  "nodes": {
    "borderWidth": 0,
    "borderWidthSelected": 0,
    "color": "rgba(166, 33, 117)",
    "opacity": null,
    "font": {
      "size": 12
    },
    "scaling": {
      "min": 2,
      "max": 72,
      "label": {
        "enabled": true,
        "min": null,
        "max": null,
        "maxVisible": null,
        "drawThreshold": null
      }
    },
    "shadow": {
      "enabled": true,
      "size": 15,
      "x": 20
    },
    "shape": "circle",
    "size": 80
  },
  "edges": {
    "color": {
      "color": "rgb(230, 163, 182)",
      "opacity": 0.9
    },
    "dashes": true,
    "selfReferenceSize": null,
    "selfReference": {
      "angle": 0.7853981633974483
    },
    "smooth": {
      "forceDirection": "none"
    }
  },
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -2000
    },
    "minVelocity": 0.75
  }
}"""
)
#net.toggle_physics(True) # выключение физического взаимодействия
net2.show('mygraph2.html')

<Figure size 1080x720 with 0 Axes>

In [13]:
d.index = df_filtered.index
df_filtered['d'] = d
df_filtered['dd'] = dd

In [14]:
df_filtered.index = description
sorted_df = df_filtered.sort_values(by='d', ascending=False).head(10)
newdf_d = sorted_df.drop(sorted_df.iloc[:, 0 : 56], axis = 1 )
newdf_d.style.background_gradient(subset=['d'], cmap='RdPu')

,d,dd
Description,,
Mining and quarrying,0.903529,0.644353
Advertising and market research,0.886367,0.060567
Architectural and engineering activities; technical testing and analysis,0.857173,0.034581
Printing and reproduction of recorded media,0.806823,0.118449
Legal and accounting activities; activities of head offices; management consultancy activities,0.804742,0.071850
Postal and courier activities,0.776321,0.277867
Administrative and support service activities,0.751119,0.228494
Activities auxiliary to financial services and insurance activities,0.738725,0.120529
"Other professional, scientific and technical activities; veterinary activities",0.737265,0.352215


In [15]:
sorted_df = df_filtered.sort_values(by='dd', ascending=False).head(10)
newdf_dd = sorted_df.drop(sorted_df.iloc[:, 0 : 56], axis = 1 )
newdf_dd.style.background_gradient(subset=['dd'], cmap='Purples')

,d,dd
Description,,
"Sewerage; waste collection, treatment and disposal activities; materials recovery; remediation activities and other waste management services",0.495968,1.145428
Mining and quarrying,0.903529,0.644353
Wholesale and retail trade and repair of motor vehicles and motorcycles,0.275989,0.643817
Manufacture of electrical equipment,0.281895,0.501909
"Manufacture of food products, beverages and tobacco products",0.143050,0.454964
"Other professional, scientific and technical activities; veterinary activities",0.737265,0.352215
Manufacture of basic metals,0.536983,0.343625
Manufacture of machinery and equipment n.e.c.,0.195619,0.283375
Postal and courier activities,0.776321,0.277867
